# Desafío - Máquinas de Soporte Vectorial

- Para realizar este desafío debes haber estudiado previamente todo el material disponibilizado correspondiente a la unidad.
- Una vez terminado el desafío, comprime la carpeta que contiene el desarrollo de los requerimientos solicitados y sube el .zip en el LMS.
- Desarrollo desafío:
    - El desafío se debe desarrollar de manera Individual
    - Para la realización del desafío necesitarás apoyarte del archivo Apoyo Desafío - Máquinas de Soporte Vectorial.

## Requerimientos
Para esta sesión trabajaremos con la base de datos sobre cáncer mamario de Wisconsin. El objetivo es desarrollar un Clasificador mediante Máquinas de Soporte de Vectores que predica de forma adecuada en base a una serie de atributos sobre la composición del núcleo de una célula mamaria. Para más detalles técnicos asociados a la base de datos, pueden hacer click en el <a href="https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.names">link</a>.

### Ejercicio 1: Preparar el ambiente de trabajo
- Importe todas las librerías a utilizar.
- Fije los parámetros de los gráficos con `plt.Rcparams`.
- Excluya las columnas `id` y `Unnamed: 32` del set de datos.
- Codifique el vector objetivo `diagnosis` a dato numérico para poder procesarlo posteriormente.

In [1]:
# Utiliza esta celda para importar las librerías y el set de datos
# importamos las librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
# import func as gfx

#sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC

labelencoder = LabelEncoder()
warnings.simplefilter('ignore')

In [2]:
# Utiliza esta celda para explorar las frecuencias del vector objetivo

In [3]:
# Utiliza esta celda para codificar el vector objetivo, asignando 1 a la clase mayoritaria y -1 a la minoritaria

### Ejercicio 2: Visualizando la distribución de los atributos
- Para cada uno de los atributos, grafique los histogramas condicional a cada clase del vector objetivo.
- Agregue las medias correspondientes y reporte a grandes rasgos cuáles son los atributos con una mayor similitud en la distribución.

In [4]:
# Utiliza esta celda para graficar los histogramas solicitados

**Comentarios**

### Ejercicio 3: Estimando el porcentaje de overlap en los atributos
- Parte de las virtudes de las Máquinas de Soporte Vectorial es la capacidad de lidiar con clases no separables mediante el proceso de kernelización. Resulta que un aspecto importante que muchas veces se obvia es medir la noseparabilidad de los atributos, condicional a cada clase del vector objetivo.
- El procedimiento para estimar el rango de noseparabilidad entre clases se implementa en Python de la siguiente manera:
```python
def histogram_overlap(df, attribute, target, perc=100):
    # get lower bound
    empirical_lower_bound = np.floor(df[attribute].min())
    
    # get upper bound
    empirical_upper_bound = np.ceil(df[attribute].max())
    
    # preserve histograms
    tmp_hist_holder = dict()
    
    # for each target class
    tar_values = df[target].unique()
    for unique_value in tar_values:
        # get histogram
        tmp, _ = np.histogram(
            df[df[target] == unique_value][attribute],   # for a specific attribute
            bins=perc,   # define percentage
            range=[empirical_lower_bound, empirical_upper_bound]   # limit empirical range for comparison
        )
        
        # append to dict
        tmp_hist_holder[f"h_{unique_value}"] = tmp
        
    get_minima = np.minimum(
        tmp_hist_holder[f"h_{tar_values[0]}"],
        tmp_hist_holder[f"h_{tar_values[1]}"]
    )
    
    intersection = np.true_divide(
        np.sum(get_minima),
        np.sum(tmp_hist_holder[f"h_{tar_values[0]}"])
    )
    
    return intersection
```
- La intersección devolverá el porcentaje de comunalidad entre ambas clases, donde mayores niveles indican una mayor comunalidad.
- Utilizando la función, generará un data frame donde almacenará el nombre del atributo y su porcentaje. Ordene este data frame de forma descendente y preserve.

In [5]:
# Utiliza esta celda para definir la función entregada
def histogram_overlap(df, attribute, target, perc=100):
    # get lower bound
    empirical_lower_bound = np.floor(df[attribute].min())
    
    # get upper bound
    empirical_upper_bound = np.ceil(df[attribute].max())
    
    # preserve histograms
    tmp_hist_holder = dict()
    
    # for each target class
    tar_values = df[target].unique()
    for unique_value in tar_values:
        # get histogram
        tmp, _ = np.histogram(
            df[df[target] == unique_value][attribute],   # for a specific attribute
            bins=perc,   # define percentage
            range=[empirical_lower_bound, empirical_upper_bound]   # limit empirical range for comparison
        )
        
        # append to dict
        tmp_hist_holder[f"h_{unique_value}"] = tmp
        
    get_minima = np.minimum(
        tmp_hist_holder[f"h_{tar_values[0]}"],
        tmp_hist_holder[f"h_{tar_values[1]}"]
    )
    
    intersection = np.true_divide(
        np.sum(get_minima),
        np.sum(tmp_hist_holder[f"h_{tar_values[0]}"])
    )
    
    return intersection

In [6]:
# Utiliza esta celda para generar el DatFrame solicitado

### Ejercicio 4: Selección del modelo por GridSearchCV
- Entrene una serie de modelos SVC con los siguientes hiper parámetros:
    - `C: [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]`
    - `gamma: [0.0000001, 0.0001, 0.001, 0.01, 0.1, 1, 10]`
    - Validaciones cruzadas: 10
    
- Genere un heatmap en base a los puntajes estimados con `GridSearchCV`. _Tip_: Vea cómo acceder a la llave `mean_test_score` en el diccionario `cv_results_`.
- Reporte en qué rango de cada hiper parámetro el modelo presenta un desempeño eficiente. Reporte la mejor combinación de hiper parámetros y el desempeño en la muestra de entrenamiento.

### Digresión: Un par de elementos a considerar en la implementación de GridSearchCV
Si trabajamos con `sklearn.model_selection.GridSearchCV`, tan solo haciendo la división en dos muestras es suficiente, incorporando los conjuntos `X_train` y `y_train` a nuestro objeto instanciado y preservando `X_test` e `y_test` como una muestra de validación externa. Si tenemos un archivo de testing externo, se recomienda no hacer división.

-  El objeto creado con `sklearn.model_selection.GridSearchCV` sigue la misma funcionalidad de cualquier método de estimación de scikit-learn, con los pasos de Instanciar y Entrenar. Este objeto tendrá muchos elementos a considerar:
    - `sklearn.model_selection.GridSearchCV.cv_results_` devolverá un diccionario donde las llaves representarán distintas métricas y los valores representarán el desempeño de cada modelo.
    - `split`: Indicará la métrica específica en cada validación cruzada y combinación de hiper parámetros.
    - `time`: Indicará el tiempo de ejecución en cada modelo.
    - Por lo general trabajaremos con `mean_test_score` y `mean_train_score` que representa la media de CV para cada combinación de hiper parámetros.
    - `sklearn.model_selection.GridSearchCV.best_estimator_` devuelve un modelo listo para entrenar con la mejor combinación de hiper parámetros.
    - `sklearn.model_selection.GridSearchCV.best_score_` devuelve el desempeño promedio del modelo en el testing interno. Si es un problema de clasificación devolverá Accuracy, si es un problema de regresión devolverá MSE.

In [7]:
# Utiliza esta celda para dividir las muestras

In [8]:
# Utiliza esta celda para definir el pipeline (use kernel rbf)

In [9]:
# Utiliza esta celda para definir la grilla de parámetros

In [10]:
# Utiliza esta celda para definir el objeto GridSearchCV

In [11]:
# Utiliza esta celda para entrenar la grilla

In [12]:
# Utiliza esta celda para definir la función para reportar los heatmaps de hiperparámetros
def report_heatmaps(cv_trained):
    param1 = tuple(cv_trained.param_grid.keys())[0]
    param2 = tuple(cv_trained.param_grid.keys())[1]
    
    # Mejores parámetros subconjuntos para test
    plt.subplot(1, 2, 1)
    sns.heatmap(
        cv_trained.cv_results_['mean_test_score'].reshape(len(cv_trained.param_grid[param1]), len(cv_trained.param_grid[param2])),
        cmap='Blues',
        annot=True,
        xticklabels=cv_trained.param_grid[param2],
        yticklabels=cv_trained.param_grid[param1],
        cbar=False
    )
    
    plt.ylabel(param1)
    plt.xlabel(param2)
    plt.title('Test CV')
    
    # Mejores parámetros subconjuntos para train
    plt.subplot(1, 2, 2)
    sns.heatmap(
        cv_trained.cv_results_['mean_train_score'].reshape(len(cv_trained.param_grid[param1]), len(cv_trained.param_grid[param2])),
        cmap='Blues',
        annot=True,
        xticklabels=cv_trained.param_grid[param2],
        yticklabels=cv_trained.param_grid[param1],
        cbar=False
    )
    
    plt.ylabel(param1)
    plt.xlabel(param2)
    plt.title('Train CV')
    plt.tight_layout()

In [13]:
# Utiliza esta celda para llamar la función definida

In [14]:
# Utiliza esta celda para reportar los mejores hiperparámetros y mejor puntaje

### Ejercicio 5: Validación del modelo en el Test set sample
- Genere las predicciones del Test set sample en base a la mejor combinación de hiperparámetros.
- Genere un reporte con las métricas de desempeño clásicas para los modelos de clasificación.
- Comente en qué casos el modelo presenta un desempeño deficiente.

In [15]:
# Utiliza esta celda para reportar las métricas del mejor modelo

**Comentarios**